# Loading the train dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
#Loading the train dataset
train_df = pd.read_csv("../input/multiclass-train/my_train_set.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
def sub_num(text):
    #substitutinging numbers
    text = re.sub('[0-9]+', 'numbr', text)
    return text

In [ ]:
train_df['clean_text'] = train_df['clean_text'].apply(str).apply(lambda x: sub_num(x))

In [ ]:
train_df.head()

In [ ]:
train_df['review_len'].describe()

# **Bi-directional Long Short-Term Memory**



# Try this next
https://machinelearningmastery.com/predict-sentiment-movie-reviews-using-deep-learning/
https://www.kaggle.com/vikashrajluhaniwal/sentiment-analysis-using-lstm-in-keras-93-acc
https://www.kaggle.com/shubhams9k96/sequence-classification-85-acc
https://www.kaggle.com/eriche523/bidirectional-lstm-in-keras
https://www.kaggle.com/imdevskp/imdb-review-classification-lstm-gru-cnn-glove

# Building Model

In [ ]:
import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence
from collections import Counter
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


import decimal
import tensorflow as tf
import re
from keras.models import load_model


avg = 'weighted'
num_classes = 4
ROUND_PREC = 1
NUM_EPOCHS = 200  # Will use early stopping to determine num. of epochs
encoder = LabelEncoder()
plot_labels = []


https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

Notes on 
# Keras.Callback:
- Keras supports the early stopping of training via a callback called EarlyStopping.This callback allows you to specify the performance measure to monitor, the trigger, and once triggered, it will stop the training process.
- The “monitor” allows you to specify the performance measure to monitor in order to end training.
- Often, the first sign of no further improvement may not be the best time to stop training. This is because the model may coast into a plateau of no improvement or even get slightly worse before getting much better.We can account for this by adding a delay to the trigger in terms of the number of epochs on which we would like to see no improvement. This can be done by setting the “patience” argument.

# Keras.Checkpointing:
The EarlyStopping callback will stop training once triggered, but the model at the end of training may not be the model with best performance on the validation dataset.
- An additional callback is required that will save the best model observed during training for later use. This is the ModelCheckpoint callback. 
> tf.keras.callbacks.ModelCheckpoint  when used, helps to periodically save your model during training.
- The callback will save the model to file, which requires that a path and filename be specified via the first argument 
> mc = ModelCheckpoint('best_model.h5')
- The preferred loss function to be monitored can be specified via the monitor argument. Also,  we are interested in only the very best model observed during training and this can be achieved by setting the SAVE_BEST_ONLY = TRUE.
>  mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
- You can save your model by calling the save() function on the model and specifying the filename.
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

# Keras.Callback. on_epoch_end
https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback

# Keras.CSVLogger
Callback that streams epoch results to a CSV file

# Chckepoint example

https://www.kaggle.com/vikramtiwari/tf2-tutorials-keras-save-and-restore-models

# Task 1

In [ ]:
from keras.preprocessing.text import Tokenizer
top_words = 8000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['clean_text'])

X_train = tokenizer.texts_to_sequences(train_df['clean_text'])
#X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1 

print(vocab_size)

In [ ]:
# #Trying the use the 3 gram to see if my model perfromance increases
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(ngram_range=(5,3)) 
# vectorized = vectorizer.fit_transform(train_df['clean_text']).toarray()
# vectorized.shape

In [ ]:
print(train_df.Text[2])

In [ ]:
print(train_df.clean_text[2])

In [ ]:
#print(vectorized[2].shape)

In [ ]:
print(X_train[2])

In [ ]:
from keras.preprocessing.sequence import pad_sequences
#X = sequence.pad_sequences(train_vector, maxlen=550)  # padding the training set
X = pad_sequences(X_train, padding='post', maxlen=500)
predictor_set = train_df['ratings']
Y = keras.utils.np_utils.to_categorical(predictor_set)


print('\n\t\t\t\t.............................Task 1 begins ................................')

print('\nChecking the different ratios of ratings in the predictor set: {} '.format(Counter(predictor_set)))

#Checking the different labels
encoder.fit(predictor_set)
labels = []
for i in range(0, 4):
    labels.append(encoder.inverse_transform([i])[0])
print('\nThe different lables are: ', labels)

#Splitting the data into training and validation set( 70 : 30)
print('\nSplitting our dataset into training and validatin set...')
train_x, x_val, train_y, y_val = train_test_split(X, Y, stratify = Y, test_size = 0.2, random_state = 42)
print('\nshape of x_val:', x_val.shape)
print('shape of y_val:', y_val.shape)

print('\n\t\t\t\t..............................Task 1 compleeted ............................')


In [ ]:
print(X.shape[1])

# Initializing our TPU

In [ ]:
import tensorflow as tf
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
max_seq_len = 500
lstm_dim  = 220 #256 better, 150 best
batch_size = 200  #best is 200 lstm_dim =200 (69,)
dropout = 0.6 #thi is the best dropout  
emb_dim  = 100    #try 64, 32 was 100
learning_rate = 0.0002 #changed form 0.01 then 0.001
num_classes = 4
loss = 'categorical_crossentropy'
metric = 'categorical_accuracy'
    

In [ ]:
from keras.layers import SpatialDropout1D
def build_model():#Training
    print('Training...')

    model = Sequential()
    
    #model.add(Embedding(vocab_size, emb_dim))
    model.add(Embedding(top_words, emb_dim, input_length=max_seq_len))
    
    #model.add(SpatialDropout1D(0.7))

    model.add(Bidirectional(LSTM(lstm_dim, dropout=dropout, recurrent_dropout=dropout)))
    #model.add(LSTM(lstm_dim, dropout=dropout, recurrent_dropout=dropout))

    model.add(Dense(num_classes))

    model.add(Activation('softmax'))

    return (model)

In [ ]:
with tpu_strategy.scope():
    model = build_model()
     
#metric = 'categorical_accuracy'eval_metrics = ['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy']
cur_optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=cur_optimizer, metrics=['categorical_accuracy']) #we will be monitoring the categorical accuracy

In [ ]:
model.summary()

In [ ]:
# Call backs for early stopping (https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min')

checkpoint_path = "weights_base.best.hdf5"  #Changed this
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=1)

history = model.fit(train_x, train_y, batch_size=batch_size, 
                        epochs=200, validation_split = 0.2, callbacks=[early_stop,checkpoint])


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

In [ ]:
history=history.history

In [ ]:
#Determing the different classes in our dataset
import itertools
num_classes = 4

def diff_classes (num_classes):
    for i in range(0, num_classes):
        plot_labels.append(i)
    return(plot_labels)


plot_classes = diff_classes(num_classes)

def compute_confusion_matrix(cm, labels , title='plot confusion matrix', cmap=plt.cm.Blues):
        
        fig, ax = plt.subplots(figsize=(5,5))

        plt.imshow(cm, cmap=cmap )
        plt.title(title)
        tick_marks = np.arange(len(labels))
        plt.xticks(tick_marks, labels, rotation=45)
        plt.yticks(tick_marks, labels)

        fmt = 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.show();

In [ ]:
from sklearn.metrics import f1_score
def model_evaluate(validation_data = ()):
    fs = []
    x_val, y_val = validation_data

    y_pred = np.asarray(model.predict(x_val)) # getting for one epoch
    y_gold = y_val

    #assert len(y_pred) == len(y_gold)

    y_pred_final, y_gold_final = ([] for _ in range(2))

    for gold, pred in zip(y_gold, y_pred):
        max_i_pred = (np.argsort(pred)[::-1])[0]  #then sorting the result per epoch and fininding the max_pred for that epoch
        max_pred_score = pred[max_i_pred]      #fininding the score for the prediction
        max_i_gold = (np.argsort(gold)[::-1])[0]
        max_gold_score = gold[max_i_gold]

        y_pred_final.append(max_i_pred)

        y_gold_final.append(max_i_gold)
        
#     #his will help see the mispredictions   
#     pred_data = pd.DataFrame(y_pred)
#     pred_data.to_csv("predictions.csv", header=True, index=False)


    fs = f1_score(y_gold_final, y_pred_final, average=None)
    #fs.append(';'.join(str(round(100*x, ROUND_PREC)) for x in fs.tolist()))

    #defining and plotting our confusion matrix

    print('The confusion matrix: ')
    cnf_matrix = confusion_matrix(y_gold_final, y_pred_final)
    plt.figure()

    compute_confusion_matrix(cnf_matrix, labels=labels,
                          title='plot_Confusion_matrix')


    #Gettign the classification report
    print('The classification report: ')
    target_names = ['class_' + i for i in [str(i) for i in list(plot_classes)]]
    print(classification_report(y_gold_final, y_pred_final, target_names=target_names))


    print('metrics used are:\n f1_score = {}'.format(fs))


In [ ]:
model_evaluate(validation_data = (x_val, y_val))

In [ ]:
best_epoch_metrics = []

# Determine if best loss is last epoch or the one before it
#epoch number is 0based
best_epoch_num = (len(history['val_loss'])-2)  if (history['val_loss'][len(history['val_loss'])-1] > history['val_loss'][len(history['val_loss'])-2]) else (len(history['val_loss'])-1) 
best_epoch_metrics.append(str(best_epoch_num))
print ('The best Epoch:', best_epoch_num )


'''#saving the best metrics for the best epoch
for metric in eval_metrics:
    metric_vals = history[metric]
    best_epoch_metrics.append(str(round(float(100*metric_vals[best_epoch_num]), ROUND_PREC)))

print("The best epoch metrics are:", best_epoch_metrics)
'''

In [ ]:
#stop here for now#

# TESTING ON TEST_SET

In [ ]:
from keras.models import load_model
test_model = load_model('weights_base.best.hdf5')


In [ ]:
# Importing the test set initially saved
test_df = pd.read_csv("../input/testing/my_test_set.csv")

In [ ]:
test_df.head()

In [ ]:
test_df['clean_text'] = test_df['clean_text'].apply(str).apply(lambda x: sub_num(x))

In [ ]:
test_df['review_len'].describe()

In [ ]:
# test_features = test_df['clean_text']
# test_predictor = test_df['ratings']

In [ ]:
tokenizer.fit_on_texts(test_df['clean_text'])

test_data = tokenizer.texts_to_sequences(test_df['clean_text'])
#X_test = tokenizer.texts_to_sequences(sentences_test)


In [ ]:
#padding the test_features
test_features = pad_sequences(test_data, padding='post', maxlen=500)
test_labels = keras.utils.np_utils.to_categorical(test_df['ratings'])


In [ ]:
def model_test_evaluate(validation_data = ()):
    fs = []
    x_val, y_val = validation_data
    ##################################
    y_pred = np.asarray(model.predict(x_val))
    y_gold = y_val
    #assert len(y_pred) == len(y_gold)

    y_pred_final, y_gold_final = ([] for _ in range(2))

    for gold, pred in zip(y_gold, y_pred):
        max_i_pred = (np.argsort(pred)[::-1])[0]  #then sorting the result per epoch and fininding the max_pred for that epoch
        max_pred_score = pred[max_i_pred]      #fininding the score for the prediction
        max_i_gold = (np.argsort(gold)[::-1])[0]
        max_gold_score = gold[max_i_gold]

        y_pred_final.append(max_i_pred)
        y_gold_final.append(max_i_gold)
        
        #his will help see the mispredictions   
    pred_test_data = pd.DataFrame(y_pred)
    pred_test_data.to_csv("predictions_test.csv", header=True, index=False)


    fs = f1_score(y_gold_final, y_pred_final, average=None)
   #fs.append(';'.join(str(round(100*x, ROUND_PREC)) for x in fs.tolist()))

    #defining and plotting our confusion matrix

    print('The confusion matrix: ')
    cnf_matrix = confusion_matrix(y_gold_final, y_pred_final)
    plt.figure()

    compute_confusion_matrix(cnf_matrix, labels=labels,
                          title='plot_Confusion_matrix')


    #Gettign the classification report
    print('The classification report: ')
    target_names = ['class_' + i for i in [str(i) for i in list(plot_classes)]]
    print(classification_report(y_gold_final, y_pred_final, target_names=target_names))

    print('metrics used are:\n f1_score = {}'.format(fs))
    return pred_test_data

In [ ]:
model_test_evaluate(validation_data = (test_features, test_labels));

In [ ]:
#Load the predicted set and calculate the misprediction rates
  ''' y_pred_scores = model.predict(x_val)

    y_pred_labels, y_pred = ([] for _ in range(2))

    for i in range(len(y_pred_scores)):
        cur_pred = y_pred_scores[i]
        max_i = np.argsort(cur_pred)[::-1][0]
        max_score = cur_pred[max_i]

        y_pred.append(max_i)

        pred_label = encoder.inverse_transform(max_i)
        y_pred_labels.append(pred_label)

        y_val = encoder.transform(y_val)
        assert len(y_pred_labels) == len(y_val)

        pred_data = pd.DataFrame(y_pred)
        pred_data.to_csv("predictions.csv", header=True, index=False)
    '''
    #################################


In [ ]:
#Load the predicted set
#y_test_pred = pd.read_csv("")
test_df['Predicted_label'] = pred_test_data
test_mispredictions = test_df[['clean_text',"ratings", 'Predicted_label', 'Originally']][test_labels != pred_test_data]

test_mispredictions.shape

In [ ]:
print("percentage of mispredictions from test_data: {} %". format((len(test_mispredictions)/len(test_df))*100)) 

stop here for now, 
https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
https://machinelearningmastery.com/introduction-to-regularization-to-reduce-overfitting-and-improve-generalization-error/
https://machinelearningmastery.com/batch-normalization-for-training-of-deep-neural-networks/
https://towardsdatascience.com/dont-overfit-how-to-prevent-overfitting-in-your-deep-learning-models-63274e552323

to read for tomorrow

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

https://www.kaggle.com/questions-and-answers/92749

https://www.kaggle.com/artgor/eda-and-lstm-cnn/notebook